# **Example of *Paper***

In this notebook we provide the code that can be used to obtain the results from the *paper* in Section *6* to get the first basis elements of the almost-commuting basis for generic operators $L_3$ and $L_5$ of order 3 and 5, respectively.

This implementation is done for these particular cases, and they show intermidiate steps. The complete process can be seen in the method `almost_commuting_wilson` in `dalgebra.commutators.almost_commuting`.

In [1]:
import sys
sys.path.insert(0, "..")
from dalgebra import *
%display latex

In [71]:
import re
from functools import reduce
def poly_to_latex(poly, name_u=None,name_y=None):
    r'''
        Transform a polynomial to a proper LaTeX representation.
        
        We assume the polynomial has variables `u_i`, `c_{i,j}` and `y_i` where the variables `u_i` and `y_i` are differential variables.
        The differential variables are given with `y_i^{(j)}` --  "yi_j".
        The constant variables are given with `c_{i,j}` -- "c_i_j".
        
        This method also guarantees the removal of "*" for multiplications and the "+ (-...)".
    '''
    if not isinstance(poly, str):
        poly = str(poly)
        
    patterns = list()
    
    patterns.append((r"\*", "")) # We remove the multiplication symbol
    patterns.append((r"\+ \(-([^\)]+)\)", lambda M: f"- {M.groups()[0]}")) # We remove the + (-number)
    patterns.append((r"\(-([^\)]+)\)", lambda M: f"-{M.groups()[0]}")) # We remove the + (-number)
    patterns.append((r"(\d+)/(\d+)", lambda M: f"\\frac\u007b{M.groups()[0]}\u007d\u007b{M.groups()[1]}\u007d"))
    patterns.append((r"c_(\d+)_(\d+)", lambda M: f"c_\u007b{M.groups()[0]},{M.groups()[1]}\u007d")) # Rewritting constant variables
    ## Rewriting differential variables
    patterns.append((r"([u])(\d+)_0([^\d]?)", lambda M: f"{M.groups()[0] if name_u == None else name_u}_\u007b{M.groups()[1]}\u007d{M.groups()[2]}"))
    patterns.append((r"([u])(\d+)_1([^\d]?)", lambda M: f"{M.groups()[0] if name_u == None else name_u}_\u007b{M.groups()[1]}\u007d'{M.groups()[2]}"))
    patterns.append((r"([u])(\d+)_2([^\d]?)", lambda M: f"{M.groups()[0] if name_u == None else name_u}_\u007b{M.groups()[1]}\u007d''{M.groups()[2]}"))
    patterns.append((r"([u])(\d+)_3([^\d]?)", lambda M: f"{M.groups()[0] if name_u == None else name_u}_\u007b{M.groups()[1]}\u007d'''{M.groups()[2]}"))
    patterns.append((r"([u])(\d+)_(\d+)", lambda M: f"{M.groups()[0] if name_u == None else name_u}_\u007b{M.groups()[1]}\u007d^\u007b({M.groups()[2]})\u007d"))
    patterns.append((r"([u])(\d+)", lambda M: f"{M.groups()[0] if name_u == None else name_u}_\u007b{M.groups()[1]}\u007d"))
    patterns.append((r"([y])(\d+)_0([^\d]?)", lambda M: f"{M.groups()[0] if name_y == None else name_y}_\u007b{M.groups()[1]}\u007d{M.groups()[2]}"))
    patterns.append((r"([y])(\d+)_1([^\d]?)", lambda M: f"{M.groups()[0] if name_y == None else name_y}_\u007b{M.groups()[1]}\u007d'{M.groups()[2]}"))
    patterns.append((r"([y])(\d+)_2([^\d]?)", lambda M: f"{M.groups()[0] if name_y == None else name_y}_\u007b{M.groups()[1]}\u007d''{M.groups()[2]}"))
    patterns.append((r"([y])(\d+)_3([^\d]?)", lambda M: f"{M.groups()[0] if name_y == None else name_y}_\u007b{M.groups()[1]}\u007d'''{M.groups()[2]}"))
    patterns.append((r"([y])(\d+)_(\d+)", lambda M: f"{M.groups()[0] if name_y == None else name_y}_\u007b{M.groups()[1]}\u007d^\u007b({M.groups()[2]})\u007d"))
    patterns.append((r"([y])(\d+)", lambda M: f"{M.groups()[0] if name_y == None else name_y}_\u007b{M.groups()[1]}\u007d"))
    patterns.append((r"z_(\d+)", lambda M: f"\\partial^\u007b{M.groups()[0]}\u007d"))
    
    return reduce(lambda p, q : re.sub(q[0], q[1], p), patterns, poly)

def print_pseudo(op):
    return poly_to_latex(op[0]) + r" + \left(" + poly_to_latex(op[1]) + r"\right)\partial^{-1}"

In [72]:
from dalgebra.commutators.almost_commuting import hierarchy, recursion
import logging
logging.getLogger("dalgebra.commutators").setLevel(logging.INFO)

## Case of order 3

In [73]:
## Setting up the framework to work
R.<u2,u3,y2,y3,y4,y5,z> = DifferentialPolynomialRing(QQ)
u = (1, 0, u2, u3)
y = (1, 0, y2, y3, y4, y5)
L = sum(z[i]*u[3-i] for i in range(4))
## Showing L
LatexExpr(poly_to_latex(L))

u_{2}\partial^{1} + u_{3}\partial^{0} + \partial^{3}

In [74]:
## Creating the \tilde{P}_i polynomials
PT = [1, z[1]]
while(len(PT) <= 5): # creating up to P_6(U;Y)
    PT += [PT[-1](z=z[1]) + y[len(PT)]*z[0]]

## Showing the PT
for (i,p) in enumerate(PT):
    show(LatexExpr(r"\tilde{P}_{" + str(i) + r"}(U,Y) \longrightarrow " + poly_to_latex(p)))

\tilde{P}_{0}(U,Y) \longrightarrow 1

\tilde{P}_{1}(U,Y) \longrightarrow \partial^{1}

\tilde{P}_{2}(U,Y) \longrightarrow y_{2}\partial^{0} + \partial^{2}

\tilde{P}_{3}(U,Y) \longrightarrow y_{2}\partial^{1} + y_{3}\partial^{0} + \partial^{3}

\tilde{P}_{4}(U,Y) \longrightarrow y_{2}\partial^{2} + y_{3}\partial^{1} + y_{4}\partial^{0} + \partial^{4}

\tilde{P}_{5}(U,Y) \longrightarrow y_{2}\partial^{3} + y_{3}\partial^{2} + y_{4}\partial^{1} + y_{5}\partial^{0} + \partial^{5}

#### **1. Computing the differential systems**

In [6]:
lie_brackets = [L.lie_bracket(P, "z") for P in PT]

In [8]:
diff_systems = [tuple([bracket.coefficient(z[i]) for i in range(L.order(z)-1, bracket.order(z)+1)]) for bracket in lie_brackets]

##### The system for $m = 1$

In [9]:
bracket = lie_brackets[1]
LatexExpr(r"[L_3, \tilde{P}_1(U)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^1+\left(-u_{3}'\right)

In [10]:
print(_)

[L_3, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^1+\left(-u_{3}'\right)


In [11]:
## There are no equations for the almost commuting of z[1]
##  --> trivial: it is the unique monic operator of order 1 in normal form
diff_systems[1]

()

##### The system for $m=2$

In [12]:
bracket = lie_brackets[2]
LatexExpr(r"[L_3, \tilde{P}_2(U, y_2)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 3y_{2}'\right)\partial^2+\left(-u_{2}'' - 2u_{3}' + 3y_{2}''\right)\partial^1+\left(u_{2}y_{2}' - u_{3}'' + y_{2}'''\right)

In [13]:
print(_)

[L_3, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 3y_{2}'\right)\partial^2+\left(-u_{2}'' - 2u_{3}' + 3y_{2}''\right)\partial^1+\left(u_{2}y_{2}' - u_{3}'' + y_{2}'''\right)


In [14]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[2]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [15]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 3$

In [16]:
bracket = lie_brackets[3]
LatexExpr(r"[L_3, \tilde{P}_3(U, y_2, y_3)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 3y_{2}'\right)\partial^3+\left(-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^2+\left(-u_{2}'y_{2} + u_{2}y_{2}' - u_{2}''' - 3u_{3}'' + y_{2}''' + 3y_{3}''\right)\partial^1+\left(u_{2}y_{3}' - u_{3}'y_{2} - u_{3}''' + y_{3}'''\right)

In [17]:
print(_)

[L_3, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 3y_{2}'\right)\partial^3+\left(-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^2+\left(-u_{2}'y_{2} + u_{2}y_{2}' - u_{2}''' - 3u_{3}'' + y_{2}''' + 3y_{3}''\right)\partial^1+\left(u_{2}y_{3}' - u_{3}'y_{2} - u_{3}''' + y_{3}'''\right)


In [18]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[3]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-3u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [19]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-3u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 4$

In [20]:
bracket = lie_brackets[4]
LatexExpr(r"[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 3y_{2}'\right)\partial^4+\left(-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^3+\left(-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^2+\left(-u_{2}''y_{2} - u_{2}'y_{3} + u_{2}y_{3}' - 2u_{3}'y_{2} - u_{2}^{(4)} - 4u_{3}''' + y_{3}''' + 3y_{4}''\right)\partial^1+\left(u_{2}y_{4}' - u_{3}''y_{2} - u_{3}'y_{3} - u_{3}^{(4)} + y_{4}'''\right)

In [21]:
print(_)

[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 3y_{2}'\right)\partial^4+\left(-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^3+\left(-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^2+\left(-u_{2}''y_{2} - u_{2}'y_{3} + u_{2}y_{3}' - 2u_{3}'y_{2} - u_{2}^{(4)} - 4u_{3}''' + y_{3}''' + 3y_{4}''\right)\partial^1+\left(u_{2}y_{4}' - u_{3}''y_{2} - u_{3}'y_{3} - u_{3}^{(4)} + y_{4}'''\right)


In [22]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[4]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-4u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [23]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-4u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 5$

In [24]:
bracket = lie_brackets[5]
LatexExpr(r"[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 3y_{2}'\right)\partial^5+\left(-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^4+\left(-3u_{2}'y_{2} + u_{2}y_{2}' - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^3+\left(-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'\right)\partial^2+\left(-u_{2}'''y_{2} - u_{2}''y_{3} - u_{2}'y_{4} + u_{2}y_{4}' - 3u_{3}''y_{2} - 2u_{3}'y_{3} - u_{2}^{(5)} - 5u_{3}^{(4)} + y_{4}''' + 3y_{5}''\right)\partial^1+\left(u_{2}y_{5}' - u_{3}'''y_{2} - u_{3}''y_{3} - u_{3}'y_{4} - u_{3}^{(5)} + y_{5}'''\right)

In [25]:
print(_)

[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 3y_{2}'\right)\partial^5+\left(-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^4+\left(-3u_{2}'y_{2} + u_{2}y_{2}' - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^3+\left(-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'\right)\partial^2+\left(-u_{2}'''y_{2} - u_{2}''y_{3} - u_{2}'y_{4} + u_{2}y_{4}' - 3u_{3}''y_{2} - 2u_{3}'y_{3} - u_{2}^{(5)} - 5u_{3}^{(4)} + y_{4}''' + 3y_{5}''\right)\partial^1+\left(u_{2}y_{5}' - u_{3}'''y_{2} - u_{3}''y_{3} - u_{3}'y_{4} - u_{3}^{(5)} + y_{5}'''\right)


In [26]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[5]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'&= 0\\
	-3u_{2}'y_{2} + u_{2}y_{2}' - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-5u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [27]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'&= 0\\
	-3u_{2}'y_{2} + u_{2}y_{2}' - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-5u_{2}' + 3y_{2}'&= 0
\end{array}\right.


#### **2. Computing the basis of $W(L_5)$**

In [28]:
W = R.weight_func([2,3,0,0,0,0,0], [1])
T = [W.homogeneous_monomials(i) for i in range(6)]

In [29]:
for i in range(6):
    show(LatexExpr(r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}"))

\mathcal{T}_{0} = \{1\}

\mathcal{T}_{1} = \{\}

\mathcal{T}_{2} = \{u_{2}\}

\mathcal{T}_{3} = \{u_{2}',u_{3}\}

\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2\}

\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}'u_{2},u_{2}u_{3}^{(0)}\}

In [30]:
print(LatexExpr(
    r"\begin{itemize}" + "\n\t" + r"\item $" + ("\n\t" + r"\item $").join(
        r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}$"
    for i in range(6)) + "\n" + r"\end{itemize}"))

\begin{itemize}
	\item $\mathcal{T}_{0} = \{1\}$
	\item $\mathcal{T}_{1} = \{\}$
	\item $\mathcal{T}_{2} = \{u_{2}\}$
	\item $\mathcal{T}_{3} = \{u_{2}',u_{3}\}$
	\item $\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2\}$
	\item $\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}'u_{2},u_{2}u_{3}^{(0)}\}$
\end{itemize}


In [31]:
## We create now the constants
C = C = [[],[]] + [[f"c_{i}_{j}" for j in range(len(T[i]))] for i in range(2,6)]
RR = R.add_constants(*sum((c for c in C), []))
RwC = RR.base()
C = [[],[]] + [[RwC(f"c_{i}_{j}") for j in range(len(T[i]))] for i in range(2,6)]

In [32]:
y_subs = {f"y{i}" : sum(c*t for (c,t) in zip(C[i], T[i])) for i in range(2,6)}
print(LatexExpr(
    r"\begin{array}{rcl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(k) + r" & \rightarrow & " + poly_to_latex(v) for (k,v) in y_subs.items()) + "\n" + r"\end{array}"))

\begin{array}{rcl}
	y_{2} & \rightarrow & c_{2,0}u_{2}\\
	y_{3} & \rightarrow & c_{3,0}u_{2}' + c_{3,1}u_{3}\\
	y_{4} & \rightarrow & c_{4,2}u_{2}^2 + c_{4,0}u_{2}'' + c_{4,1}u_{3}'\\
	y_{5} & \rightarrow & c_{5,2}u_{2}'u_{2} + c_{5,3}u_{2}u_{3}^{(0)} + c_{5,0}u_{2}''' + c_{5,1}u_{3}''
\end{array}


In [33]:
## We compute the linear system for each $m$
lin_systems = [sum(([RwC.wrapped(c) for c in el(**y_subs).coefficients()] for el in diff_systems[m]), []) for m in range(6)]

In [34]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[2]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{2,0} & = -2
\end{array}\right.

In [35]:
print(_)

\left\{\begin{array}{rl}
	3c_{2,0} & = -2
\end{array}\right.


In [36]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[3]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{2,0} + 3c_{3,0} & = -3\\
	3c_{3,1} & = -3\\
	3c_{2,0} & = -3
\end{array}\right.

In [37]:
print(_)

\left\{\begin{array}{rl}
	3c_{2,0} + 3c_{3,0} & = -3\\
	3c_{3,1} & = -3\\
	3c_{2,0} & = -3
\end{array}\right.


In [38]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[4]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	-c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -4\\
	3c_{3,1} + 3c_{4,1} & = -6\\
	3c_{2,0} + 3c_{3,0} & = -6\\
	3c_{3,1} & = -4\\
	3c_{2,0} & = -4
\end{array}\right.

In [39]:
print(_)

\left\{\begin{array}{rl}
	-c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -4\\
	3c_{3,1} + 3c_{4,1} & = -6\\
	3c_{2,0} + 3c_{3,0} & = -6\\
	3c_{3,1} & = -4\\
	3c_{2,0} & = -4
\end{array}\right.


In [40]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[5]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3c_{2,0} + c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,1} + 3c_{5,3} & = 0\\
	-3c_{2,0} + c_{3,1} + 3c_{5,3} & = 0\\
	c_{3,0} + 3c_{4,0} + 3c_{5,0} & = -5\\
	c_{3,1} + 3c_{4,1} + 3c_{5,1} & = -10\\
	-2c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -10\\
	3c_{3,1} + 3c_{4,1} & = -10\\
	3c_{2,0} + 3c_{3,0} & = -10\\
	3c_{3,1} & = -5\\
	3c_{2,0} & = -5
\end{array}\right.

In [41]:
print(_)

\left\{\begin{array}{rl}
	-3c_{2,0} + c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,1} + 3c_{5,3} & = 0\\
	-3c_{2,0} + c_{3,1} + 3c_{5,3} & = 0\\
	c_{3,0} + 3c_{4,0} + 3c_{5,0} & = -5\\
	c_{3,1} + 3c_{4,1} + 3c_{5,1} & = -10\\
	-2c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -10\\
	3c_{3,1} + 3c_{4,1} & = -10\\
	3c_{2,0} + 3c_{3,0} & = -10\\
	3c_{3,1} & = -5\\
	3c_{2,0} & = -5
\end{array}\right.


In [42]:
solutions = [ideal(system).groebner_basis() if len(system) > 0 else [] for system in lin_systems]

In [43]:
for i in range(len(solutions)):
    show(i, LatexExpr(r"\qquad\longrightarrow\qquad"), [LatexExpr(poly_to_latex(sol)) for sol in solutions[i]])

0 \qquad\longrightarrow\qquad []

1 \qquad\longrightarrow\qquad []

2 \qquad\longrightarrow\qquad [c_{2,0} - \frac{2}{3}]

3 \qquad\longrightarrow\qquad [c_{2,0} - 1, c_{3,0}, c_{3,1} - 1]

4 \qquad\longrightarrow\qquad [c_{2,0} - \frac{4}{3},
 c_{3,0} - \frac{2}{3},
 c_{3,1} - \frac{4}{3},
 c_{4,0} - \frac{2}{9},
 c_{4,1} - \frac{2}{3},
 c_{4,2} - \frac{2}{9}]

5 \qquad\longrightarrow\qquad [c_{2,0} - \frac{5}{3},
 c_{3,0} - \frac{5}{3},
 c_{3,1} - \frac{5}{3},
 c_{4,0} - \frac{10}{9},
 c_{4,1} - \frac{5}{3},
 c_{4,2} - \frac{5}{9},
 c_{5,0},
 c_{5,1} - \frac{10}{9},
 c_{5,2},
 c_{5,3} - \frac{10}{9}]

#### **3. Computing the hierarchy equations**

In [44]:
y_subs_true = [dict(),dict()] + [{f"y{i}" : sum(QQ(RwC.wrapped(c).reduce(solutions[m]))*t for (c,t) in zip(C[i], T[i])) for i in range(2,m+1)} for m in range(2,6)]

In [45]:
H = [[lie_brackets[m].coefficient(z[i])(**y_subs_true[m]) for i in range(L.order(z)-1)] for m in range(6)]

##### m = 0

In [46]:
m = 0
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3) = & 0\\
	H_{0,1}(u_2,u_3) = & 0
\end{array}\right.

In [47]:
print(_)

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3) = & 0\\
	H_{0,1}(u_2,u_3) = & 0
\end{array}\right.


##### m = 1

In [48]:
m=1
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3) = & -u_{3}'\\
	H_{1,1}(u_2,u_3) = & -u_{2}'
\end{array}\right.

In [49]:
print(_)

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3) = & -u_{3}'\\
	H_{1,1}(u_2,u_3) = & -u_{2}'
\end{array}\right.


##### m = 2

In [50]:
m=2
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3) = & \frac{2}{3}u_{2}'u_{2} + \frac{2}{3}u_{2}''' - u_{3}''\\
	H_{2,1}(u_2,u_3) = & u_{2}'' - 2u_{3}'
\end{array}\right.

In [51]:
print(_)

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3) = & \frac{2}{3}u_{2}'u_{2} + \frac{2}{3}u_{2}''' - u_{3}''\\
	H_{2,1}(u_2,u_3) = & u_{2}'' - 2u_{3}'
\end{array}\right.


##### m = 3

In [52]:
m=3
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3) = & 0\\
	H_{3,1}(u_2,u_3) = & 0
\end{array}\right.

In [53]:
print(_)

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3) = & 0\\
	H_{3,1}(u_2,u_3) = & 0
\end{array}\right.


##### m = 4

In [54]:
m=4
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3) = & \frac{4}{9}u_{2}'u_{2}^2 + \frac{2}{3}u_{2}'''u_{2} + \frac{4}{3}u_{2}''u_{2}' - \frac{2}{3}u_{2}'u_{3}^{(1)} - \frac{2}{3}u_{2}u_{3}'' - \frac{4}{3}u_{3}'u_{3} + \frac{2}{9}u_{2}^{(5)} - \frac{1}{3}u_{3}^{(4)}\\
	H_{4,1}(u_2,u_3) = & \frac{2}{3}u_{2}''u_{2} + \frac{2}{3}u_{2}'^2 - \frac{4}{3}u_{2}'u_{3} - \frac{4}{3}u_{2}u_{3}' + \frac{1}{3}u_{2}^{(4)} - \frac{2}{3}u_{3}'''
\end{array}\right.

In [55]:
print(_)

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3) = & \frac{4}{9}u_{2}'u_{2}^2 + \frac{2}{3}u_{2}'''u_{2} + \frac{4}{3}u_{2}''u_{2}' - \frac{2}{3}u_{2}'u_{3}^{(1)} - \frac{2}{3}u_{2}u_{3}'' - \frac{4}{3}u_{3}'u_{3} + \frac{2}{9}u_{2}^{(5)} - \frac{1}{3}u_{3}^{(4)}\\
	H_{4,1}(u_2,u_3) = & \frac{2}{3}u_{2}''u_{2} + \frac{2}{3}u_{2}'^2 - \frac{4}{3}u_{2}'u_{3} - \frac{4}{3}u_{2}u_{3}' + \frac{1}{3}u_{2}^{(4)} - \frac{2}{3}u_{3}'''
\end{array}\right.


##### m = 5

In [56]:
m=5
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3) = & \frac{10}{9}u_{2}'u_{2}u_{3}^{(0)} + \frac{5}{9}u_{2}^2u_{3}' + \frac{10}{9}u_{2}'''u_{3} + \frac{20}{9}u_{2}''u_{3}' + \frac{5}{3}u_{2}'u_{3}'' + \frac{5}{9}u_{2}u_{3}''' - \frac{5}{3}u_{3}''u_{3} - \frac{5}{3}u_{3}'^2 + \frac{1}{9}u_{3}^{(5)}\\
	H_{5,1}(u_2,u_3) = & \frac{5}{9}u_{2}'u_{2}^2 + \frac{5}{9}u_{2}'''u_{2} + \frac{5}{9}u_{2}''u_{2}' + \frac{5}{3}u_{2}''u_{3} + \frac{5}{3}u_{2}'u_{3}^{(1)} - \frac{10}{3}u_{3}'u_{3} + \frac{1}{9}u_{2}^{(5)}
\end{array}\right.

In [57]:
print(_)

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3) = & \frac{10}{9}u_{2}'u_{2}u_{3}^{(0)} + \frac{5}{9}u_{2}^2u_{3}' + \frac{10}{9}u_{2}'''u_{3} + \frac{20}{9}u_{2}''u_{3}' + \frac{5}{3}u_{2}'u_{3}'' + \frac{5}{9}u_{2}u_{3}''' - \frac{5}{3}u_{3}''u_{3} - \frac{5}{3}u_{3}'^2 + \frac{1}{9}u_{3}^{(5)}\\
	H_{5,1}(u_2,u_3) = & \frac{5}{9}u_{2}'u_{2}^2 + \frac{5}{9}u_{2}'''u_{2} + \frac{5}{9}u_{2}''u_{2}' + \frac{5}{3}u_{2}''u_{3} + \frac{5}{3}u_{2}'u_{3}^{(1)} - \frac{10}{3}u_{3}'u_{3} + \frac{1}{9}u_{2}^{(5)}
\end{array}\right.


#### **Comparing with literature**

In [93]:
# q_1 -> u_2; q_0 -> u_3
subs_to_paper = {"u3": (1/2*u[2][1] + u[3])}
str_to_paper = lambda el : LatexExpr(poly_to_latex(el).replace("u_{2}", "q_{1}").replace("u_{3}", "q_{0}"))

In [94]:
# [P_r, L_3] = 3 f_n+1' \partial + 3/2 f_n+1'' + 3g_n+1'
# r = 3n+1 or 3n+2
# So in our case, for r=3n+l
# 3 f_{n+1}' = -H_{r, 1}; 3/2 f_{n+1}'' + 3g_{n+1}' = -H_{r, 0}
# Solving this we get
# f_{n+1} = -\int (1/3) H_{r,1}
# g_{n+1} = -\int (1/3)H_{r, 0} - 1/2 f_{n+1}'

In [100]:
# r = 1 (mod 3)
# f_1 = \int (1/3) H_{1,1}
# g_1 = \int (1/3) H_{1,0} - 1/2 f_1
f_1 = -((1/3)*H[1][1](**subs_to_paper)).inverse_operation(0)
g_1 = -((1/3)*H[1][0](**subs_to_paper)).inverse_operation(0) - (1/2)*f_1

In [101]:
str_to_paper(f_1)

\frac{1}{3}q_{1}

In [102]:
str_to_paper(H[1][0])

-q_{0}'

In [103]:
str_to_paper(g_1)

\frac{1}{6}q_{1}' - \frac{1}{6}q_{1} + \frac{1}{3}q_{0}

In [ ]:
# r = 1 (mod 3)
# f_2 = \int (1/3) H_{4,1}
# g_2 = \int (1/3) H_{4,0} - 1/2 f_2

In [ ]:
# r = 2 (mod 3)
# f_1 = \int (1/3) H_{2,1}
# g_1 = \int (1/3) H_{2,0} - 1/2 f_1

In [ ]:
# r = 2 (mod 3)
# f_2 = \int (1/3) H_{5,1}
# g_2 = \int (1/3) H_{5,0} - 1/2 f_2

## Case of order 5

In [ ]:
## Setting up the framework to work
R.<u2,u3,u4,u5,y2,y3,y4,y5,y6,z> = DifferentialPolynomialRing(QQ)
u = (1, 0, u2, u3, u4, u5)
y = (1, 0, y2, y3, y4, y5, y6)
L = sum(z[i]*u[5-i] for i in range(6))
## Showing L
LatexExpr(poly_to_latex(L))

In [ ]:
## Creating the \tilde{P}_i polynomials
PT = [1, z[1]]
while(len(PT) <= 6): # creating up to P_6(U;Y)
    PT += [PT[-1](z=z[1]) + y[len(PT)]*z[0]]

## Showing the PT
for (i,p) in enumerate(PT):
    show(LatexExpr(r"\tilde{P}_{" + str(i) + r"}(U,Y) \longrightarrow " + poly_to_latex(p)))

#### **1. Computing the differential systems**

In [ ]:
lie_brackets = [L.lie_bracket(P, "z") for P in PT]

In [ ]:
diff_systems = [tuple([bracket.coefficient(z[i]) for i in range(L.order(z)-1, bracket.order(z)+1)]) for bracket in lie_brackets]

##### The system for $m = 1$

In [ ]:
bracket = lie_brackets[1]
LatexExpr(r"[L_5, \tilde{P}_1(U)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

In [ ]:
print(_)

In [ ]:
## There are no equations for the almost commuting of z[1]
##  --> trivial: it is the unique monic operator of order 1 in normal form
diff_systems[1]

##### The system for $m=2$

In [ ]:
bracket = lie_brackets[2]
LatexExpr(r"[L_5, \tilde{P}_2(U, y_2)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

In [ ]:
print(_)

In [ ]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[2]) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### The system for $m = 3$

In [ ]:
bracket = lie_brackets[3]
LatexExpr(r"[L_5, \tilde{P}_3(U, y_2, y_3)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

In [ ]:
print(_)

In [ ]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[3]) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### The system for $m = 4$

In [ ]:
bracket = lie_brackets[4]
LatexExpr(r"[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

In [ ]:
print(_)

In [ ]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[4]) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### The system for $m = 5$

In [ ]:
bracket.coefficient(z[5])

In [ ]:
bracket = lie_brackets[5]
LatexExpr(r"[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") + "\n\t" for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

In [ ]:
print(_)

In [ ]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[5]) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### The system for $m = 6$

In [ ]:
bracket = lie_brackets[6]
LatexExpr(r"[L_5, \tilde{P}_6(U, Y)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") + "\n\t" for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

In [ ]:
print(_)

In [ ]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[6]) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

#### **2. Computing the basis of $W(L_5)$**

In [ ]:
W = R.weight_func([2,3,4,5,0,0,0,0,0,0], [1])
T = [W.homogeneous_monomials(i) for i in range(7)]

In [ ]:
for i in range(7):
    show(LatexExpr(r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}"))

In [ ]:
print(LatexExpr(
    r"\begin{itemize}" + "\n\t" + r"\item $" + ("\n\t" + r"\item $").join(
        r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}$"
    for i in range(7)) + "\n" + r"\end{itemize}"))

In [ ]:
## We create now the constants
RwC = DifferentialRing(PolynomialRing(QQ, sum(([f"c_{i}_{j}" for j in range(len(T[i]))] for i in range(2,7)), [])))
C = [[],[]] + [[RwC(f"c_{i}_{j}") for j in range(len(T[i]))] for i in range(2,7)]

In [ ]:
y_subs = {f"y{i}" : sum(c*t for (c,t) in zip(C[i], T[i])) for i in range(2,7)}
print(LatexExpr(
    r"\begin{array}{rcl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(k) + r" & \rightarrow & " + poly_to_latex(v) for (k,v) in y_subs.items()) + "\n" + r"\end{array}"))

In [ ]:
## We compute the linear system for each $m$
lin_systems = [sum(([RwC.wrapped(c) for c in el(**y_subs).coefficients()] for el in diff_systems[m]), []) for m in range(7)]

In [ ]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[2]) + "\n" r"\end{array}\right.")

In [ ]:
print(_)

In [ ]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[3]) + "\n" r"\end{array}\right.")

In [ ]:
print(_)

In [ ]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[4]) + "\n" r"\end{array}\right.")

In [ ]:
print(_)

In [ ]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[5]) + "\n" r"\end{array}\right.")

In [ ]:
print(_)

In [ ]:
len(lin_systems[6])

In [ ]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[6]) + "\n" r"\end{array}\right.")

In [ ]:
print(_)

In [ ]:
solutions = [ideal(system).groebner_basis() if len(system) > 0 else [] for system in lin_systems]

In [ ]:
for i in range(len(solutions)):
    show(i, LatexExpr(r"\qquad\longrightarrow\qquad"), [LatexExpr(poly_to_latex(sol)) for sol in solutions[i]])

#### **3. Computing the hierarchy equations**

In [ ]:
y_subs_true = [dict(),dict()] + [{f"y{i}" : sum(QQ(RwC.wrapped(c).reduce(solutions[m]))*t for (c,t) in zip(C[i], T[i])) for i in range(2,m+1)} for m in range(2,7)]

In [ ]:
H = [[lie_brackets[m].coefficient(z[i])(**y_subs_true[m]) for i in range(L.order(z)-1)] for m in range(7)]

##### m = 0

In [ ]:
m = 0
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### m = 1

In [ ]:
m=1
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### m = 2

In [ ]:
m=2
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### m = 3

In [ ]:
m=3
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### m = 4

In [ ]:
m=4
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### m = 5

In [ ]:
m=5
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

##### m = 6

In [ ]:
m=6
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

In [ ]:
print(_)

In [ ]:
order = lambda P : max([sum(o for o in el.orders() if o != -1) for el in P.monomials()])

In [ ]:
for i in range(4):
    print(i, (order(H[6][i]), H[6][i].degree(), len(H[6][i].monomials())))

#### **Computations with the hierarchy?**

In [ ]:
recursion(5)

In [104]:
p = hierarchy(5,2)[0]
R = p.parent()
RwC = R.add_constants("a","b")
u = [None, None] + list(p.parent().gens()[:-1])
z = p.parent().gens()[-1]
p

2/5*u2_3*u2_0 + 2/5*u2_2*u3_0 + 2/5*u2_1*u4_0 + 2/5*u2_5 - u5_2

In [115]:
elements = set()
while(len(elements) < 100):
    elements.add(R.random_element(5,5))
names = {str(el) for el in elements}

In [116]:
%%time 
for _ in range(10):
    T0 = [R(el) for el in elements]

CPU times: user 584 µs, sys: 6 µs, total: 590 µs
Wall time: 592 µs


In [117]:
%%time 
for _ in range(10):
    T1 = [RwC(el) for el in elements]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/qh82kz/sage/local/var/lib/sage/venv-python3.10.5/lib/python3.10/site-packages/IPython/core/magics/execution.py", line 1316, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "<timed exec>", line 2, in <listcomp>
  File "/home/qh82kz/git/dalgebra/notebooks/../dalgebra/dpolynomial/dpolynomial_ring.py", line 572, in __call__
    res = super().__call__(*args, **kwds)
  File "sage/structure/parent.pyx", line 897, in sage.structure.parent.Parent.__call__
    return mor._call_(x)
  File "sage/structure/coerce_maps.pyx", line 156, in sage.structure.coerce_maps.DefaultConvertMap_unique._call_
    return C._element_constructor(x)
  File "/home/qh82kz/git/dalgebra/notebooks/../dalgebra/dpolynomial/dpolynomial_ring.py", line 345, in _element_constructor_
    p = sum(self.base()(c)*self(str(t)) for (c,t) in zip(x.coefficients(), x.monomials()))
  File "/home/qh82kz/git/dalgebra/notebooks/../dalgebra/dpolynomial/

In [118]:
%%time 
for _ in range(10):
    T4 = [el.change_ring(RwC.base()) for el in elements]

KeyboardInterrupt: 

In [119]:
%%time 
for _ in range(10):
    T2 = [R(el) for el in names]

CPU times: user 13.4 s, sys: 0 ns, total: 13.4 s
Wall time: 13.4 s


In [120]:
%%time 
for _ in range(10):
    T3 = [RwC(el) for el in names]

CPU times: user 37.7 s, sys: 50 ms, total: 37.7 s
Wall time: 37.7 s


In [ ]:
from sage.categories.pushout import pushout

In [ ]:
z[10].divides(p)

In [ ]:
R = (z[10] + u[5][2] + p).polynomial().parent()

In [ ]:
z[10] + u[5][2] + p

In [ ]:
R(z[10].polynomial()).divides(R(p.polynomial()))

In [ ]:
R.__class__

In [ ]:
R(p.polynomial())

In [ ]:
p.degree(u[5][7])

In [ ]:
R(p.polynomial()).degree(R(z[10].polynomial()))